In [1]:
import numpy as np
import pandas as pd
from gurobipy import *
np.random.seed(1)

In [2]:
def Corte_Entrada(i,A):
    Ce = []
    for (x,y) in A:
        if y== i:
            Ce = Ce + [(x,y)]
    return Ce

def Corte_Salida(i):
    Cs = []
    for (x,y) in A:
            if x==i:
                Cs = Cs + [(x,y)]
    return Cs

def Corte_Entrada_Nodo(i,A):
    Ce = []
    for (x,y) in A:
        if y== i:
            Ce = Ce + [x]
    return Ce

def Corte_Salida_Nodo(i,A):
    Cs = []
    for (x,y) in A:
            if x==i:
                Cs = Cs + [y]
    return Cs

## MODELO 3

In [4]:
def Trabajo(gap=0,time=3600):
    m = Model()

    # Crear las variables
    x = m.addVars([(k, m, i, j) for (k, m, i, j) in K], name="x", vtype=GRB.BINARY)
    y = m.addVars([(k, m, j) for k in range (1,n1+1) for m in M[k] for j in V_D], name="y", vtype=GRB.BINARY)
    a = m.addVars([(k, m, i, d) for k in range (1,n1+1) for m in M[k] for i in V for d in range(1,1+n2)] , name = "a", vtype = GRB.INTEGER, lb=0)
    z = m.addVars([(k, m) for k in range (1,n1+1) for m in M[k] ], name = "z", vtype = GRB.BINARY )
    u = m.addVars([(k, m, j) for k in range (1,n1+1) for m in M[k] for j in V], name="u", vtype = GRB.INTEGER, lb=1, ub= n)
    m.update()
    
    #Función Objetivo
    t_V = {(k, m, i, j): c[i,j] for(k,m,i,j) in K}

    m.setObjective(x.prod(t_V) + z.prod(c_M) , GRB.MINIMIZE)

    #Restricciones
    
    for k,b,i,j in K:
        if type(i) == str or type(j) == str:
            continue
        m.addConstr(
            (u[k,b,j]  >= u[k,b,i] + (1 + n)*x[k,b,i,j] - n)
        )
    
    
    m.addConstrs(
        ( x.sum(k, m,"O"+str(k) ,"*")  == z[k,m] for k in range(1,n1+1) for m in M[k]) 
    )

    m.addConstrs(
        quicksum(x[k, m, j[0], "D"+str(p)]  for p in range(1,n2+1) for j in Corte_Entrada("D"+str(p), At) ) == z[k, m]  for k in range(1,n1+1) for m in M[k]
    )
    
    m.addConstrs(
        ( x.sum(k, m,"*" ,i)  == y[k, m, i]  for k in range(1,n1+1) for m in M[k] for i in V) 
    ) 
    
    m.addConstrs(
        ( x.sum(k, m,"*","D"+str(p) )  == y[k, m, "D"+str(p)] for k in range(1,n1+1) for m in M[k] for p in range(1,n2+1)) 
    )
    
    m.addConstrs(
        ( x.sum(k, m, "*",s)  == x.sum(k, m ,s,"*") for k in range(1,n1+1) for m in M[k] for s in V)
    )

#     m.addConstrs(
#         ( x.sum(k, "*","O"+str(k),"*") <= len(M[k]) for k in range(1,n1+1))
#     )
    
    m.addConstrs(
        (a.sum(k, m, "*", p) <= Dem[p]*y[k, m, "D"+str(p)] for k in range(1,n1+1) for m in M[k] for p in range(1,1+n2))
    )
    
    m.addConstrs(
        (a.sum("*", "*", i, "*") <= OF[i] for i in V )
    )


    m.addConstrs(
        (a.sum(k, m,"*", "*") <= Q[k, m]  for k in range(1,n1+1) for m in M[k])
    )

    m.addConstrs(
       (a[k, m, i, p]<= OF[i]*y[k, m, i] for p in range(1,n2+1)  for k in range(1,n1+1) for m in M[k] for i in V )
    )

    m.addConstrs(
        (a.sum("*", "*", "*", p) == Dem[p] for p in range(1,n2+1))
    )

    m.Params.MIPGap = gap
    m.Params.TimeLimit = time
    
    m.optimize()
    par={}
    clasi = {}
    producto = {}
    
    try:
    
        if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
            par = {"Obj":m.objval, "GAP":abs(m.objval-m.ObjBoundC) /abs(m.objval), "Time": m.Runtime}


        if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
        # Recuperar los valores de las variables
            vx = m.getAttr('x', x)
            print('Arcos seleccionados:')
            for (k,m1,i,j) in K:
                if vx[k,m1,i,j] >= 0.99:
                    print('Deposito {} usa el auto{}:= {} -> {}'.format(k,m1, i, j))
                    if ("O"+str(k)+"-"+str(m1) in clasi.keys() )== False:
                        clasi.update({"O"+str(k)+"-"+str(m1) : [(i,j)]})
                    else:
                        clasi["O"+str(k)+"-"+str(m1)]  = clasi["O"+str(k)+"-"+str(m1)] + [(i,j)]


        print("-"*20)


        if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
             for k in range (1,n1+1):
                    for m1 in M[k]:
                        for i in V :
                            for d in range(1,1+n2):
                                if a[k,m1,i,d].x>0:
                                    c_prod = a[k,m1,i,d].x
                                    if (("O"+str(k),m1) in producto.keys() )== False:
                                            producto.update({("O"+str(k),m1): [(i, c_prod,d)] })
                                    else:
                                        producto[("O"+str(k),m1)] = producto[("O"+str(k),m1)] + [(i, c_prod,d)]
                                    print("El auto {} del deposito {} recoge {} del producto en {} para {}".format(m1,k,c_prod,i,d))

        if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
            for k in range (1,n1+1):
                for m1 in M[k]: 
                    for j in V_D:
                        if y[k,m1,j].x>0.99:
                            print("el auto {} del deposito {} visita {}".format(m1, k, j))
    except:
        print("TIME LIMIT!")
        
        
    return clasi, producto, par

## MODELO 4

In [5]:
def Trabajo1 (gap=0, time = 3600):
    np.random.seed(1)
    m = Model()
    # Crear las variables
    print("CREANDO VARIABLES")
    x = m.addVars([(k, m, i, j) for (k, m, i, j) in K], name="x", vtype=GRB.BINARY)
    y = m.addVars([(k, m, j) for k in range (1,n1+1) for m in M[k] for j in V_D], name="y", vtype=GRB.BINARY)
    a = m.addVars([(k, m, i, d) for k in range (1,n1+1) for m in M[k] for i in V for d in range(1,1+n2)] , name = "a", vtype = GRB.INTEGER, lb=0)
    u = m.addVars( [(k, m, j) for k in range (1,n1+1) for m in M[k] for j in V_D], name="u", vtype = GRB.INTEGER, lb=1, ub= n)
    m.update()
    
    #Función Objetivo
    t_V = {(k, m, i, j): c[i,j] for(k,m,i,j) in K}
    C_M ={ (k, m, "O"+str(k), i): c_M[k,m] for k in range(1,n1+1) for m in M[k] for i in V}
    
    print("CREANDO FUNCION OBJ")
    m.setObjective(x.prod(t_V) + x.prod(C_M) , GRB.MINIMIZE)

    #Restricciones
    
    print("CREANDO RESTRICCIONES")
    
    for k,b,i,j in K:
        if type(i) == str or type(j) == str:
            continue
        m.addConstr(
            (u[k,b,j]  >= u[k,b,i] + (1 + n)*x[k,b,i,j] - n)
        )
    
    m.addConstrs(
        ( x.sum(k, m,"O"+str(k) ,"*")  <= 1 for k in range(1,n1+1) for m in M[k]) 
    )

    m.addConstrs(
        quicksum(x[k, m, j[0], "D"+str(p)]  for p in range(1,n2+1) for j in Corte_Entrada("D"+str(p), At) ) <=1  for k in range(1,n1+1) for m in M[k]
    )
    
    m.addConstrs(
        ( x.sum(k, m,"*" ,i)  == y[k, m, i]  for k in range(1,n1+1) for m in M[k] for i in V) 
    ) 
    
    m.addConstrs(
        ( x.sum(k, m,"*","D"+str(p) )  == y[k, m, "D"+str(p)] for k in range(1,n1+1) for m in M[k] for p in range(1,n2+1)) 
    )
    
    m.addConstrs(
        ( x.sum(k, m, "*",s)  == x.sum(k, m ,s,"*") for k in range(1,n1+1) for m in M[k] for s in V)
    )

#     m.addConstrs(
#         ( x.sum(k, "*","O"+str(k),"*") <= len(M[k]) for k in range(1,n1+1))
#     )
    
    m.addConstrs(
        (a.sum(k, m, "*", p) <= Dem[p]*y[k, m, "D"+str(p)] for k in range(1,n1+1) for m in M[k] for p in range(1,1+n2))
    )
    
    m.addConstrs(
        (a.sum("*", "*", i, "*") <= OF[i] for i in V )
    )


    m.addConstrs(
        (a.sum(k, m,"*", "*") <= Q[k, m]  for k in range(1,n1+1) for m in M[k])
    )

    m.addConstrs(
        (a[k, m, i, p]<= OF[i]*y[k, m, i] for p in range(1,n2+1)  for k in range(1,n1+1) for m in M[k] for i in V )
    )

    m.addConstrs(
        (a.sum("*", "*", "*", p) == Dem[p] for p in range(1,n2+1))
    )

    m.Params.MIPGap = gap
    m.Params.TimeLimit= time
    print("OPTIMIZE")
    
    
    print("-"*20)

    clasi = {}
    par = {}
    producto = {}
    print("GUARDANDO SOLUCIONES")
    
    try:
        
        if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
            par = {"Obj":m.objval, "GAP":abs(m.objval-m.ObjBoundC) /abs(m.objval), "Time": m.Runtime}
        # Recuperar los valores de las variables
            vx = m.getAttr('x', x)
            print('Arcos seleccionados:')
            for (k,m1,i,j) in K:
                if vx[k,m1,i,j] >= 0.99:
                    print('Deposito {} usa el auto{}:= {} -> {}'.format(k,m1, i, j))
                    if ("O"+str(k)+"-"+str(m1) in clasi.keys() )== False:
                        clasi.update({"O"+str(k)+"-"+str(m1) : [(i,j)]})
                    else:
                        clasi["O"+str(k)+"-"+str(m1)]  = clasi["O"+str(k)+"-"+str(m1)] + [(i,j)]


        print("-"*20)

        if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
             for k in range (1,n1+1):
                    for m1 in M[k]:
                        for i in V :
                            for d in range(1,1+n2):
                                if a[k,m1,i,d].x>0:
                                    c_prod = a[k,m1,i,d].x
                                    if (("O"+str(k),m1) in producto.keys() )== False:
                                            producto.update({("O"+str(k),m1): [(i, c_prod,d)] })
                                    else:
                                        producto[("O"+str(k),m1)] = producto[("O"+str(k),m1)] + [(i, c_prod,d)]
                                    print("El auto {} del deposito {} recoge {} del producto en {} para {}".format(m1,k,c_prod,i,d))

        if m.status == GRB.Status.TIME_LIMIT or  m.status == GRB.Status.OPTIMAL:
            for k in range (1,n1+1):
                for m1 in M[k]: 
                    for j in V_D:
                        if y[k,m1,j].x>0.99:
                            print("el auto {} del deposito {} visita {}".format(m1, k, j))
    except:
        print("TIME LIMIT!")
    
    return clasi, producto, par

## LECTURA Y RESOLUCIÓN DE LAS 52 INTANCIAS 

In [8]:

from IPython.display import Markdown


df_instancias = pd.read_excel("InstanciasFinal.xlsx",sheet_name= "Sheet1", header=0, index_col=[0,1,2])


Instancias = {5: {11: [17], 14: [17], 17: [2], 18: [6]},
 10: {11: [10], 15: [4], 17: [9], 21: [11]},
 15: {6: [5], 7: [6], 18: [7], 22: [9]},
 20: {12: [2], 13: [8], 19: [10], 24: [4]},
 25: {11: [12], 16: [9], 17: [17], 24: [17]},
 30: {10: [2], 15: [3], 20: [12], 24: [16]},
 35: {6: [15], 15: [2], 19: [5], 23: [19]},
 40: {6: [4], 7: [8], 13: [8], 15: [15]},
 45: {14: [13], 19: [9], 23: [14], 25: [15]},
 50: {13: [13], 14: [14], 16: [13], 21: [10]},
 55: {9: [6], 12: [9], 23: [9], 24: [15]},
 60: {10: [6], 17: [20], 18: [2], 23: [18]},
 65: {15: [9], 19: [12], 22: [15], 25: [19]}}
    

for j in df_instancias.columns:
    s = df_instancias[j]
    for m in list(Instancias.keys()):
        for i in list(Instancias[m].keys()):
            for k in Instancias[m][i]:
                s[m,i,k] = list([int(s[m,i,k].split(",")[0][1:]), int(s[m,i,k].split(",")[1][:-1])+1]) #mas 1 por como se generan los #aleatorios


df_primeras = df_instancias

prob = 0.5

DatosInstancias3 = {}
for n in list(Instancias.keys()):
    V = tuplelist(range(1,n+1))
    np.random.seed(1)
    a1 = {(i,j): np.random.randint(35,151) for i in V for j in V if (i!=j and np.random.random()<prob)}

    for i,j in list(a1.keys()):
        if (j,i) in list(a1.keys()) and i<j:
            a1[j,i] = a1[i,j]

    a = {**a1}
    A = list(a.keys())
    for n1 in list(Instancias[n].keys()):
        O = ["O"+str(i) for i in range(1,n1+1)]
        np.random.seed(1)
        Ao = [ (i,j) for i in O for j in V if np.random.random()<0.85 ]
        np.random.seed(1)
        b1 = {(i,j): np.random.randint(35,150) for (i,j) in Ao }
        V_O = [i for i in range(1,n+1)]+O
        
        for n2 in Instancias[n][n1]:
            display (Markdown('<span style="color: #ff0000">{},{} y {}</span> '.format(n,n1,n2)))
            np.random.seed(1)
            #Creación de D
            D = ["D"+str(i) for i in range(1,n2+1)]
            np.random.seed(1)
            Ad = [ (i,j) for i in V for j in D if np.random.random()<0.85 ]
            np.random.seed(1)
            b2 = {(i,j): np.random.randint(35,150)  for (i,j) in Ad }
            V_D = [i for i in range(1,n+1)]+D

            At,c= multidict({**a, **b1, **b2})
            #Vehículos
            np.random.seed(1)
            M = {k: [i for i in range(1,
                                                  np.random.randint(df_primeras.loc[n].loc[n1]["Rango de \nVehículo para cada deposito"][n2][0]+1,
                                                                    df_primeras.loc[n].loc[n1]["Rango de \nVehículo para cada deposito"][n2][1]+1))] for k in range(1,n1+1) }
            np.random.seed(1)
            c_M = { (k, m):  np.random.randint(350,500) for k in range(1,n1+1) for m in M[k]}
            
            
            
            
            #Creación K
            K = []
            for k in range(1,n1+1):
                for m in M[k]:
                    K = K + [(k, m, j[0], j[1]) for j in Ao if "O"+str(k) == j[0]  ]
                    K = K + [(k,m,i,j) for (i,j) in A+Ad]
                    
            #Capacidades de los vehículos
            np.random.seed(1)
            Q = {(k, m) : np.random.randint(df_primeras.loc[n].loc[n1]["Rango de \ncapacidad"][n2][0], df_primeras.loc[n].loc[n1]["Rango de \ncapacidad"][n2][1]) for k in range(1,n1+1) for m in M[k]}

            #Oferta
            np.random.seed(1)
            OF = {i: np.random.randint(df_primeras.loc[n].loc[n1]["Oferta para\n cada lugar de abastecimiento"][n2][0], df_primeras.loc[n].loc[n1]["Oferta para\n cada lugar de abastecimiento"][n2][1]) for i in V}

            #Demanda
            np.random.seed(1)
            Dem = {p :np.random.randint(df_primeras.loc[n].loc[n1]["Rango Demanda"][n2][0], df_primeras.loc[n].loc[n1]["Rango Demanda"][n2][1]) for p in range(1,n2+1) }

            print(len(At))        
            print("Modelo1")
            Ruta1, Pro1, Par1 = Trabajo()
            
            print("Modelo2")
            Ruta2, Pro2, Par2 = Trabajo1()
            
            DatosInstancias3.update({(n,n1,n2): [c,M,c_M, Q, OF, Dem, Ruta1, Pro1, Par1, Ruta2, Pro2, Par2] })


<span style="color: #ff0000">40,6 y 4</span> 

Modelo1
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10294 rows, 11854 columns and 61532 nonzeros
Model fingerprint: 0xf46e9296
Variable types: 0 continuous, 11854 integer (9854 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [4e+01, 5e+02]
  Bounds range     [1e+00, 4e+01]
  RHS range        [1e+01, 1e+02]
Presolve time: 0.13s
Presolved: 10294 rows, 11854 columns, 54791 nonzeros
Variable types: 0 continuous, 11854 integer (9854 binary)

Root relaxation: objective 3.201708e+03, 4778 iterations, 0.52 seconds (0.43 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 3201.70821    0  456          - 3201.70821      -     -    0s
     0     0 3323.06427    

  2844  1377 5414.83231   53  246 5472.00000 5387.68447  1.54%   211  306s
  2966  1441 5430.58263   77  243 5472.00000 5387.77321  1.54%   208  310s
  3232  1486 5399.36815   41  185 5472.00000 5387.77321  1.54%   200  316s
  3389  1459 5425.38038   68  340 5472.00000 5387.77321  1.54%   196  320s
  3675  1637 5393.89082   34  453 5472.00000 5387.99004  1.54%   187  328s
  3801  1681 5415.08157   52  340 5472.00000 5387.99004  1.54%   185  332s
  3884  1765 5451.46523   82  338 5472.00000 5388.06459  1.53%   184  336s
  4002  1873 5398.16486   40  345 5472.00000 5388.14920  1.53%   185  340s
  4301  2130 5396.12118   44  451 5472.00000 5388.17393  1.53%   182  348s
  4405  2214 5414.21456   64  498 5472.00000 5388.17393  1.53%   184  353s
  4497  2311 5429.46030  101  446 5472.00000 5388.27029  1.53%   185  358s
  4618  2414 5390.02815   36  441 5472.00000 5388.27029  1.53%   186  364s
  4736  2548 5411.89834   53  388 5472.00000 5388.40098  1.53%   188  369s
  4909  2603 5389.52445  

 18194  8875 5396.28329   64  412 5449.00000 5389.28205  1.10%   189 1095s
 18264  9028 5398.96121   71  494 5449.00000 5389.28205  1.10%   191 1107s
 18556  9072 5411.86843   93  339 5449.00000 5389.28205  1.10%   193 1120s
 18753  9173 5442.53145  119  281 5449.00000 5389.28205  1.10%   195 1132s
 19072  9315 5393.10024   56  378 5449.00000 5389.28205  1.10%   197 1146s
 19380  9578 5398.09499   57  302 5449.00000 5389.28205  1.10%   199 1160s
 19794  9662 5439.50903  151  352 5449.00000 5389.28205  1.10%   199 1172s
 20099 10004 5429.40970   71  450 5449.00000 5389.40956  1.09%   201 1187s
 20593 10112 5406.54114   99  395 5449.00000 5389.45233  1.09%   201 1210s
 20940  9997 5414.25560  155  382 5449.00000 5389.45233  1.09%   200 1215s
 20944 10000 5416.89965  116  636 5449.00000 5389.45233  1.09%   200 1220s
 20949 10003 5435.72713   85  477 5449.00000 5389.45233  1.09%   200 1225s
 20962 10012 5405.70235   63  451 5449.00000 5389.45233  1.09%   200 1230s
 20971 10018 5420.97949  

 26888 11218 5398.52564   79  403 5414.00000 5389.45233  0.45%   211 1833s
 27234 11221 5398.14225  131  431 5414.00000 5389.45233  0.45%   210 1839s
 27401 11446 5395.50878   81  329 5414.00000 5389.51109  0.45%   210 1845s
 27759 11539 5391.97514   75  334 5414.00000 5389.63030  0.45%   210 1852s
 28067 11680 5400.12500   83  440 5414.00000 5389.71076  0.45%   210 1860s
 28380 11786 5410.04397   83  260 5414.00000 5389.75223  0.45%   210 1869s
 28698 11878 5393.56419   79  352 5414.00000 5389.81731  0.45%   210 1879s
 28964 11974 5397.69327   96  453 5414.00000 5389.88151  0.45%   210 1889s
 29301 12141 5393.04653   79  454 5414.00000 5389.95562  0.44%   210 1900s
 29677 12134 5398.28198   74  245 5414.00000 5390.01266  0.44%   210 1910s
 29862 12320 5402.38306   97  385 5414.00000 5390.01666  0.44%   211 1921s
 30207 12414 5396.85467   81  247 5414.00000 5390.02110  0.44%   211 1932s
 30554 12618 5394.45966   97  424 5414.00000 5390.10034  0.44%   212 1943s
 30996 12752 5397.28623  

...

     0     0 9895.63337    0 2938          - 9895.63337      -     -  767s
     0     0 9895.91214    0 2885          - 9895.91214      -     -  827s
     0     0 9895.92623    0 2884          - 9895.92623      -     -  828s
     0     0 9950.65170    0 2370          - 9950.65170      -     -  889s
     0     0 9951.20063    0 2447          - 9951.20063      -     -  895s
     0     0 9951.22829    0 2419          - 9951.22829      -     -  896s
     0     0 10074.1134    0 2322          - 10074.1134      -     -  960s
     0     0 10080.6638    0 2416          - 10080.6638      -     -  968s
     0     0 10080.9704    0 2427          - 10080.9704      -     -  970s
     0     0 10080.9707    0 2447          - 10080.9707      -     -  971s
     0     0 10296.5362    0 2047          - 10296.5362      -     - 1039s
     0     0 10304.0561    0 1972          - 10304.0561      -     - 1058s
     0     0 10306.9052    0 1994          - 10306.9052      -     - 1066s
     0     0 10306.9840  

<span style="color: #ff0000">65,25 y 19</span> 

Modelo1
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 200358 rows, 263051 columns and 1340211 nonzeros
Model fingerprint: 0x40567b83
Variable types: 0 continuous, 263051 integer (187651 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [4e+01, 5e+02]
  Bounds range     [1e+00, 7e+01]
  RHS range        [4e+01, 1e+02]
Presolve time: 2.56s
Presolved: 200358 rows, 263051 columns, 1162362 nonzeros
Variable types: 0 continuous, 263051 integer (187651 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.388750e+03   3.545700e+11      8s
   28105    5.1560675e+03   3.886957e+02   1.017725e+11     10s
   45181    7.3631959e+03

     0     0 11318.1104    0 3105          - 11318.1104      -     - 2426s
     0     0 11318.1167    0 3115          - 11318.1167      -     - 2428s
     0     0 11425.5660    0 2646          - 11425.5660      -     - 2554s
     0     0 11461.1282    0 2786          - 11461.1282      -     - 2641s
     0     0 11461.9656    0 2772          - 11461.9656      -     - 2656s
     0     0 11462.0343    0 2797          - 11462.0343      -     - 2660s
     0     0 11462.0661    0 2786          - 11462.0661      -     - 2663s
     0     0 11567.1818    0 2821          - 11567.1818      -     - 2757s
     0     0 11568.9573    0 2792          - 11568.9573      -     - 2780s
     0     0 11569.6569    0 2904          - 11569.6569      -     - 2786s
     0     0 11569.6966    0 2856          - 11569.6966      -     - 2789s
     0     0 11651.9366    0 2677          - 11651.9366      -     - 2892s
     0     0 11667.0807    0 2853          - 11667.0807      -     - 2938s
     0     0 11667.4777  

  547207    1.1113558e+04   0.000000e+00   8.215630e+04    365s
  554527    1.1107558e+04   0.000000e+00   7.396632e+05    370s
  561603    1.1102615e+04   0.000000e+00   2.600540e+05    375s
  569411    1.1096322e+04   0.000000e+00   4.251752e+04    380s
  577219    1.1088793e+04   0.000000e+00   4.245195e+05    385s
  585271    1.1081423e+04   0.000000e+00   3.489708e+06    390s
  592347    1.1075803e+04   0.000000e+00   1.440840e+05    395s
  598935    1.1071033e+04   0.000000e+00   6.233690e+04    400s
  605523    1.1066342e+04   0.000000e+00   4.282305e+04    405s
  613819    1.1061545e+04   0.000000e+00   9.503828e+04    410s
  622115    1.1056724e+04   0.000000e+00   5.632743e+05    415s
  630899    1.1051318e+04   0.000000e+00   7.496450e+04    420s
  639195    1.1046502e+04   0.000000e+00   3.893875e+04    425s
  647979    1.1040541e+04   0.000000e+00   2.065725e+04    430s
  656519    1.1035151e+04   0.000000e+00   4.701188e+04    435s
  664815    1.1031380e+04   0.000000e+00

## IMPRIMIR INDICADORES PARA COMPARAR MODELOS

In [9]:
for i in DatosInstancias4.keys():
    print("____"*20)
    print("INSTANCIA {}".format(i))
    print("Para el modelo 1")
    print(DatosInstancias4[i][-4])
    print("Para el modelo 2")
    print(DatosInstancias4[i][-1] )

________________________________________________________________________________
INSTANCIA (40, 6, 4)
Para el modelo 1
{'Obj': 5414.0, 'GAP': 0.0039890938112784975, 'Time': 3600.1655235290527}
Para el modelo 2
{'Obj': 5428.0, 'GAP': 0.007282963157816532, 'Time': 3600.1013221740723}
________________________________________________________________________________
INSTANCIA (40, 7, 8)
Para el modelo 1
{'Obj': 9526.0, 'GAP': 0.2642192343801534, 'Time': 3600.099298477173}
Para el modelo 2
{'Obj': 9491.0, 'GAP': 0.2613236360245064, 'Time': 3600.121322631836}
________________________________________________________________________________
INSTANCIA (40, 13, 8)
Para el modelo 1
{'Obj': 9350.0, 'GAP': 0.2667286482037599, 'Time': 3600.115245819092}
Para el modelo 2
{'Obj': 9293.0, 'GAP': 0.03541367203829096, 'Time': 3600.1243476867676}
________________________________________________________________________________
INSTANCIA (40, 15, 15)
Para el modelo 1
{'Obj': 16367.0, 'GAP': 0.146667819379858

## GUARDAR SOLUCIONES

In [17]:
# open file for writing
f = open("SOLUCIONES_MODELO3y4.txt","w")

# write file
f.write( str(DatosInstancias3) )

# close file
f.close()

## COLOCACIÓN DE INDICADORES EN CADA INSTANCIA

In [18]:
df_instancias = pd.read_excel("InstanciasFinal.xlsx",sheet_name= "Sheet1", header=0, index_col=[0,1,2])
dfcolumns = [("Parametros",i) for i in df_instancias.columns]
df_instancias.columns = dfcolumns




a = pd.Series([DatosInstancias3[i][-4]["Obj"] for i in DatosInstancias3.keys()])
a.index = df_instancias.index
df_instancias[("MODELO 1","OBJ")] = a

a = pd.Series([DatosInstancias3[i][-4]["GAP"] for i in DatosInstancias3.keys()])
a.index = df_instancias.index
df_instancias[("MODELO 1","GAP")] = a

a = pd.Series([DatosInstancias3[i][-4]["Time"] for i in DatosInstancias3.keys()])
a.index = df_instancias.index
df_instancias[("MODELO 1","Tiempo")] = a

a = pd.Series([DatosInstancias3[i][-1]["Obj"] for i in DatosInstancias3.keys()])
a.index = df_instancias.index
df_instancias[("MODELO 2","OBJ")] = a

a = pd.Series([DatosInstancias3[i][-1]["GAP"] for i in DatosInstancias3.keys()])
a.index = df_instancias.index
df_instancias[("MODELO 2","GAP")] = a

a = pd.Series([DatosInstancias3[i][-1]["Time"] for i in DatosInstancias3.keys()])
a.index = df_instancias.index
df_instancias[("MODELO 2","Tiempo")] = a


df_instancias.head()

(Parametros, Rango de \nVehículo para cada deposito)  \
5  11 17                                             [2, 5]     
   14 17                                             [2, 3]     
   17 2                                              [1, 4]     
   18 6                                              [1, 3]     
10 11 10                                             [1, 2]     

         (Parametros, Rango de \ncapacidad)  \
5  11 17                         [119, 122]   
   14 17                           [98, 99]   
   17 2                            [19, 22]   
   18 6                            [44, 46]   
10 11 10                         [120, 124]   

         (Parametros, Oferta para\n cada lugar de abastecimiento)  \
5  11 17                                         [525, 527]         
   14 17                                         [545, 547]         
   17 2                                            [64, 67]         
   18 6                                          [157, 161]         
10 11 10                                         [133, 137]         

         (Parametros, Rango Demanda)  (MODELO 1, OBJ)  (MODELO 1, GAP)  \
5  11 17                  [129, 154]          18252.0         0.039620   
   14 17                  [121, 160]          18053.0         0.000000   
   17 2                   [130, 158]           6903.0         0.000000   
   18 6                   [124, 130]           9478.0         0.045017   
10 11 10                  [121, 132]           9094.0         0.000000   

          (MODELO 1, Tiempo)  (MODELO 2, OBJ)  (MODELO 2, GAP)  \
5  11 17         3600.145761          18252.0         0.000000   
   14 17         3405.008579          18053.0         0.232077   
   17 2          1253.440216           6903.0         0.000000   
   18 6          3600.116476           9478.0         0.036914   
10 11 10          339.984888           9094.0         0.000000   

          (MODELO 2, Tiempo)  
5  11 17         2719.499994  
   14 17         3600.167431  
   17 2           189.184647  
   18 6          3600.114092  
10 11 10         1312.099802

In [19]:
df_instancias.columns = pd.MultiIndex.from_tuples(df_instancias)

In [20]:
df_instancias.to_excel("RESULTADOS_INSTANCIAS_FINAL_VARIABLES_ORDENAMIENTO.xlsx")